In [5]:
import pandas as pd
import xml.etree.ElementTree as ET
from xml.dom import minidom

# Define the input and output file paths
input_tsv_file = 'bachelorthesis/P114.tsv'
output_xml_file = 'bachelorthesis/P114.xml'

# Read the Tobii eye tracker data from the TSV file using pandas
df = pd.read_csv(input_tsv_file, delimiter='\t', low_memory=False)

# Create the root element of the XML document
root = ET.Element('iTraceData')

# Iterate through rows in the dataframe and create XML elements
for index, row in df.iterrows():
    gaze_data = ET.SubElement(root, 'GazeData')

    # Assuming your TSV file has columns named 'Timestamp', 'X', and 'Y'
    timestamp = ET.SubElement(gaze_data, 'Timestamp')
    timestamp.text = str(row['Recording timestamp [ms]'])

    x_coordinate = ET.SubElement(gaze_data, 'X')
    x_coordinate.text = str(row['Gaze point X [DACS px]'])

    y_coordinate = ET.SubElement(gaze_data, 'Y')
    y_coordinate.text = str(row['Gaze point Y [DACS px]'])

# Create the XML tree and write it to the output file
tree = ET.ElementTree(root)
tree.write(output_xml_file, encoding='utf-8', xml_declaration=True)

# Beautify the output XML for readability (optional)
xml_string = ET.tostring(root, 'utf-8')
xml_pretty = minidom.parseString(xml_string).toprettyxml(indent="    ")
with open(output_xml_file, "w") as f:
    f.write(xml_pretty)

print(f"Conversion completed. Output written to {output_xml_file}")

Conversion completed. Output written to bachelorthesis/P114.xml


In [8]:
import pandas as pd
import xml.etree.ElementTree as ET
from xml.dom import minidom

# Define the input and output file paths
input_tsv_file = 'bachelorthesis/P114.tsv'
output_xml_file = 'bachelorthesis/P114.xml'

# Read the Tobii eye tracker data from the TSV file using pandas
df = pd.read_csv(input_tsv_file, delimiter='\t', low_memory=False)

# Create the root element of the XML document
root = ET.Element('core')

# Create the environment element
environment = ET.SubElement(root, 'environment')

# Add environment sub-elements
screen_size = ET.SubElement(environment, 'screen-size', width="1920", height="1080")
eye_tracker = ET.SubElement(environment, 'eye-tracker', type="Tobii Pro X3-120 EPU")
date = ET.SubElement(environment, 'date')
time = ET.SubElement(environment, 'time')
session = ET.SubElement(environment, 'session', id="114")
calibration = ET.SubElement(environment, 'calibration', id="114")
study = ET.SubElement(environment, 'study', name="Main")
researcher = ET.SubElement(environment, 'researcher', name="")
participant = ET.SubElement(environment, 'participant', id="P-114")

# Iterate through rows in the dataframe and create response elements
for index, row in df.iterrows():
    response = ET.SubElement(root, 'response', x=str(row['Gaze point X [DACS px]']), y=str(row['Gaze point Y [DACS px]']), left_x="nan", left_y="nan",
                            left_pupil_diameter="nan", left_validation="0", right_x="nan", right_y="nan",
                            right_pupil_diameter="nan", right_validation="0", tracker_time=str(row['Recording timestamp [ms]']),
                            system_time=str(row['Recording timestamp [ms]']), event_time=str(row['Recording timestamp [ms]']), fixation_id="NA")

# Create the XML tree and write it to the output file
tree = ET.ElementTree(root)
tree.write(output_xml_file, encoding='utf-8', xml_declaration=True)

# Beautify the output XML for readability (optional)
xml_string = ET.tostring(root, 'utf-8')
xml_pretty = minidom.parseString(xml_string).toprettyxml(indent="    ")
with open(output_xml_file, "w") as f:
    f.write(xml_pretty)

print(f"Conversion completed. Output written to {output_xml_file}")

Conversion completed. Output written to bachelorthesis/P114.xml


In [9]:
#Test with DB

In [63]:
import sqlite3
import pandas as pd

# Define the path to your TSV file and SQLite database
tsv_file = 'bachelorthesis/P114.tsv'
database_file = 'bachelorthesis/P114g.db3'

# Create a SQLite database and connect to it
conn = sqlite3.connect(database_file)

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Define the SQL command to create the 'gaze_attributes' table
remove_table_sql = ["participant", "fixation_run", "session", "fixation", "calibration", "calibration_point", "calibration_sample","gaze","ide_context","web_context","fixation_gaze","files"]


                    
create_table_sql = [
            "CREATE TABLE IF NOT EXISTS participant(participant_id TEXT PRIMARY KEY,session_length INTEGER);",
            "CREATE TABLE IF NOT EXISTS fixation_run(fixation_run_id INTEGER PRIMARY KEY,session_id INTEGER,date_time INTEGER,filter TEXT,FOREIGN KEY (session_id) REFERENCES session(session_id));",
            "CREATE TABLE IF NOT EXISTS session(session_id INTEGER PRIMARY KEY,participant_id TEXT,screen_width INTEGER, screen_height INTEGER,tracker_type TEXT, tracker_serial_number TEXT,session_date INTEGER, session_time INTEGER,screen_recording_start INTEGER,task_name TEXT,FOREIGN KEY (participant_id) REFERENCES participant(participant_id));",
            "CREATE TABLE IF NOT EXISTS fixation(fixation_id TEXT PRIMARY KEY,fixation_run_id INTEGER,fixation_start_event_time INTEGER,fixation_order_number INTEGER,x INTEGER,y INTEGER,fixation_target TEXT,source_file_line INTEGER, source_file_col INTEGER,token TEXT,syntactic_category TEXT,xpath TEXT,left_pupil_diameter REAL,right_pupil_diameter REAL,duration INTEGER);",
            "CREATE TABLE IF NOT EXISTS calibration(calibration_id INTEGER PRIMARY KEY);",
            "CREATE TABLE IF NOT EXISTS calibration_point(calibration_point_id TEXT,calibration_id INTEGER,calibration_x REAL,calibration_y REAL,FOREIGN KEY (calibration_id) REFERENCES calibration(calibration_id));",
            "CREATE TABLE IF NOT EXISTS calibration_sample(calibration_point_id TEXT,left_x REAL, left_y REAL,left_validation REAL,right_x REAL, right_y REAL,right_validation REAL,FOREIGN KEY (calibration_point_id) REFERENCES calibration_point(calibration_point_id));",
            "CREATE TABLE IF NOT EXISTS gaze(event_time INTEGER PRIMARY KEY,session_id INTEGER,calibration_id INTEGER,participant_id TEXT, tracker_time INTEGER, system_time INTEGER, x REAL, y REAL,left_x REAL, left_y REAL, left_pupil_diameter REAL, left_validation INTEGER,right_x REAL, right_y REAL, right_pupil_diameter REAL, right_validation INTEGER,user_left_x REAL,user_left_y REAL,user_left_z REAL,user_right_x REAL,user_right_y REAL,user_right_z REAL,FOREIGN KEY (session_id) REFERENCES session(session_id),FOREIGN KEY (calibration_id) REFERENCES calibration(calibration_id),FOREIGN KEY (participant_id) REFERENCES participant(participant_id));",
            "CREATE TABLE IF NOT EXISTS ide_context(event_time INTEGER,session_id INTEGER,time_stamp TEXT,ide_type TEXT,gaze_target TEXT,gaze_target_type TEXT,source_file_path TEXT, source_file_line INTEGER, source_file_col INTEGER,editor_line_height REAL,editor_font_height REAL, editor_line_base_x REAL, editor_line_base_y REAL,source_token TEXT,source_token_type TEXT, source_token_xpath TEXT, source_token_syntactic_context TEXT, x REAL, y REAL,FOREIGN KEY (event_time) REFERENCES gaze(event_time),FOREIGN KEY (session_id) REFERENCES session(session_id));",
            "CREATE TABLE IF NOT EXISTS web_context(event_time INTEGER,browser_type TEXT,site_name TEXT,url TEXT,tag TEXT,FOREIGN KEY (event_time) REFERENCES gaze(event_time));",
            "CREATE TABLE IF NOT EXISTS fixation_gaze(fixation_id INTEGER,event_time INTEGER,FOREIGN KEY (fixation_id) REFERENCES fixation(fixation_id),FOREIGN KEY (event_time) REFERENCES gaze(event_time));",
            "CREATE TABLE IF NOT EXISTS files(file_hash TEXT PRIMARY KEY,session_id INTEGER,file_full_path TEXT,file_type TEXT,FOREIGN KEY (session_id) REFERENCES session(session_id));",

            "CREATE INDEX IF NOT EXISTS idx_event_time_context ON ide_context(event_time);",
            "CREATE INDEX IF NOT EXISTS idx_event_time_gaze ON gaze(event_time);",
            "CREATE INDEX IF NOT EXISTS idx_fixation_id ON fixation(fixation_id);"]



# Execute the SQL command to create the table

for entry in create_table_sql:
    cursor.execute(entry)

# Read the TSV file into a Pandas DataFrame, selecting only the required columns
selected_columns = ['Gaze point X [DACS px]', 'Gaze point Y [DACS px]', 'Recording timestamp [ms]', 'Computer timestamp [ms]']
df = pd.read_csv(tsv_file, delimiter='\t', usecols=selected_columns, low_memory=False)

# Rename the columns to match the database schema
df.columns = ['event_time', 'system_time','x', 'y']
new_df = df[['event_time','x','y']]
# Define the SQL command to insert session data
session_id = 1
participant_id = 114
screen_width = '1920'
screen_height = '1080'
tracker_type = 'Tobii Pro X3-120 EPU'
tracker_serial_number = '12345'
session_date = '2023-09-18'
session_time = '12:00:00'
screen_recording_start = '12:30:00'
task_name = 'Main'

# Add the 'participant_id' and 'session_id' columns with constant values
df['participant_id'] = participant_id
df['session_id'] = session_id
df['task_name'] = task_name
df['calibration_id'] = 0
df['tracker_time'] = 0
df['left_x'] = 0
df['left_y'] = 0
df['left_pupil_diameter'] = 0
df['left_validation'] = 0
df['right_x'] = 0
df['right_y'] = 0
df['right_pupil_diameter'] = 0
df['right_validation'] = 0
df['user_left_x'] = 0
df['user_left_y'] = 0
df['user_left_z'] = 0
df['user_right_x'] = 0
df['user_right_y'] = 0
df['user_right_z'] = 0


# Reorder columns to match the table schema
#df = df[['event_time', 'session_id', 'participant_id', 'x', 'y', 'task_name', 'system_time']]


# Insert the DataFrame into the 'gaze_attributes' table
df.to_sql('gaze', conn, if_exists='replace', index=False)

cursor.execute("DELETE FROM participant")
"CREATE TABLE IF NOT EXISTS"
cursor.execute(f"INSERT INTO participant(participant_id,session_length) VALUES(\"{participant_id}\",null);")

session_insert_sql = f"""
REPLACE INTO session(session_id, participant_id, screen_width, screen_height, tracker_type, tracker_serial_number, session_date, session_time, screen_recording_start, task_name)
VALUES ({session_id}, {participant_id}, {screen_width}, {screen_height}, '{tracker_type}', '{tracker_serial_number}', '{session_date}', '{session_time}', '{screen_recording_start}', '{task_name}');
"""

# Execute the SQL command to insert session data
cursor.execute(session_insert_sql)

# Define the SQL command to insert IDE context data
print(new_df)
new_df.columns = ['event_time', 'x', 'y', ]
new_df['time_stamp'] = '1'
new_df['session_id'] = session_id
new_df['ide_type'] = 'IDE_TYPE'
new_df['gaze_target'] = 'GAZE_TARGET'
new_df['gaze_target_type'] = 'GAZE_TARGET_TYPE'
new_df['source_file_path'] = 'SOURCE_FILE_PATH'
new_df['source_file_line'] = 1
new_df['source_file_col'] = 1
new_df['editor_line_height'] = 1
new_df['editor_font_height'] = 10
new_df['editor_line_base_x'] = 1
new_df['editor_line_base_y'] = 1
new_df['source_token'] = ''
new_df['source_token_type'] = ''
new_df['source_token_xpath'] = ''
new_df['source_token_syntactic_context'] = ''

new_df.to_sql('ide_context', conn, if_exists='replace', index=False)


# Execute the SQL command to insert IDE context data
cursor.execute(ide_context_insert_sql)


# Commit changes and close the database connection
conn.commit()
conn.close()

print(f"Data imported into {database_file}")

       event_time      x       y
0               0    NaN     NaN
1              28    NaN     NaN
2              88 -102.0  1285.0
3              96 -180.0  1291.0
4             105 -180.0  1291.0
...           ...    ...     ...
38455      319915  603.0   463.0
38456      319924  622.0   455.0
38457      319932  614.0   477.0
38458      319940  620.0   478.0
38459      320334    NaN     NaN

[38460 rows x 3 columns]
Data imported into bachelorthesis/P114g.db3


C:\Users\Timon\AppData\Local\Temp\ipykernel_23288\3963206873.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['time_stamp'] = '1'
C:\Users\Timon\AppData\Local\Temp\ipykernel_23288\3963206873.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['session_id'] = session_id
C:\Users\Timon\AppData\Local\Temp\ipykernel_23288\3963206873.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [37]:
# Define the SQL command to insert IDE context data
event_time = '1'
time_stamp = 'timestamp'
ide_type = 'IDE_TYPE'
gaze_target = 'GAZE_TARGET'
gaze_target_type = 'GAZE_TARGET_TYPE'
source_file_path = 'SOURCE_FILE_PATH'
source_file_line = 'SOURCE_FILE_LINE'
source_file_col = 'SOURCE_FILE_COL'
editor_line_height = 'EDITOR_LINE_HEIGHT'
editor_font_height = 'EDITOR_FONT_HEIGHT'
editor_line_base_x = 'EDITOR_LINE_BASE_X'
editor_line_base_y = 'EDITOR_LINE_BASE_Y'
source_token = ''
source_token_type = ''
source_token_xpath = ''
source_token_syntactic_context = ''
x = 'X'
y = 'Y'

ide_context_insert_sql = f"""
INSERT INTO ide_context(
    event_time, session_id, time_stamp, ide_type, gaze_target, gaze_target_type,
    source_file_path, source_file_line, source_file_col, editor_line_height,
    editor_font_height, editor_line_base_x, editor_line_base_y, source_token,
    source_token_type, source_token_xpath, source_token_syntactic_context, x, y
)
VALUES (
    {event_time}, {session_id}, '{time_stamp}', '{ide_type}', '{gaze_target}',
    '{gaze_target_type}', '{source_file_path}', {source_file_line},
    {source_file_col}, {editor_line_height},
    {editor_font_height}, {editor_line_base_x}, {editor_line_base_y},
    {f"'{source_token}'" if source_token else 'NULL'},
    {f"'{source_token_type}'" if source_token_type else 'NULL'},
    {f"'{source_token_xpath}'" if source_token_xpath else 'NULL'},
    {f"'{source_token_syntactic_context}'" if source_token_syntactic_context else 'NULL'},
    {x}, {y}
);
"""


# Execute the SQL command to insert IDE context data
cursor.execute(ide_context_insert_sql)

OperationalError: no such column: SOURCE_FILE_LINE